# Workshop Automatants @ CentraleSupélec - CeSIA - Partie 4

- Création : 02/2025 par [Nicolas Guillard](mailto:nicolas.guillar@securite-ia.fr) - bénévole au [CeSIA](https://www.securite-ia.fr/).
- Dernière mise à jour : 05/03/2025

Créer en s'inspirant particulièrement de [Générer des noms de villes et communes françaises](https://github.com/alxndrTL/villes) par [Alexandre TL](https://www.youtube.com/@alexandretl)


## Présentation du sujet et Plan

## Indications de travail

Les éléments de ce TP :
- le présent carnet,
  
(et ceux qui seront installés grâce au script `Installation de l'environnement de travail`),
- le répertoire `./utils` et les fichiers contenus,
- le répertoire `./weights` contenant les poids des modèles utiles et ceux produits,
- le répertoire `./images` contenant les illustrations des carnets,
- le fichier de données `./villes.txt`.

## Installation de l'environnement de travail

Le script ci-dessous est destiné à installer les éléments nécessaires au fonctionnement de ce carnet.

In [ ]:
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

repo = "workshop_cs_202503"
branch = "main"
url_repo = f"https://github.com/nicolasguillard/{repo}/archive/refs/heads/{branch}.zip"
target_dir = (
  "/content"
  if IN_COLAB
  else "."
)
resources = ["utils", "weights", "images", "villes.txt"]

if not Path(f"{target_dir}/utils").exists() :
  print("=== Installation des ressources utiles à ce carnet ===")
  !wget -P {target_dir} {url_repo}
  !unzip {target_dir}/{branch}.zip -d {target_dir}
  for resource in resources:
    !mv {target_dir}/{repo}-{branch}/{resource} {target_dir}/{resource}
  !rm -rf {target_dir}/{repo}-{branch}
  !rm -f {target_dir}/{branch}.zip
  print("=== Terminé ===")

  if IN_COLAB:
    print("--- Rafraichissez au besoin la liste des fichiers à gauche si nécessaire ---")
else:
  print("Il semble que des ressources nécessaires pour ce carnet soient déjà installés :")
  for resource in resources:
    print("\t", f"./{resource}", "présent" if Path(f"{target_dir}/{resource}").exists else "absent")
  print("Pour supprimer les ressources automatiquement installées, utilisez la fonction 'remove_resources()' dans un autre bloc de code.")

def remove_resources():
  !rm -rf {target_dir}/{repo}-{branch}
  for resource in resources:
    !rm -rf {target_dir}/{resource}

## Les modules et paramétrages globaux

Tous les modules nécessaires sont importés. A moins d'un besoin spécifique, il n'y aura pas besoin de modifier le bloc de code suivant.

In [ ]:
# Modules prédéfinis et tiers
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
#Modules créés pour le projet
from utils import get_datasets, SOS, EOS, PAD, CityNameDataset
from utils import load_transformer_model, TransformerConfig, LanguageModelForSAE, sample, CharTokenizer
from utils import load_sae, AutoEncoder
from utils import clean_memory

### Device

Sélection du GPU selon l'environnement de travail

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

### Paramétrages

In [ ]:
# Retirer la limite du nombre maximal de lignes affichées dans un tableau pandas
pd.set_option('display.max_rows', None) 

In [ ]:
# Configurer le thème de seaborn
sns.set_theme(style="whitegrid")

In [ ]:
# Paramétrer les graines aléatoires
#pth_rnd_gen_device = torch.Generator(device).manual_seed(42)
if device == "cuda":
    pth_rnd_gen_device = torch.cuda.manual_seed(42)
elif device == "mps":
    pth_rnd_gen_device = torch.mps.manual_seed(42)
pth_rnd_gen_cpu = torch.manual_seed(42)
pth_rnd_gen = pth_rnd_gen_cpu if device == "cpu" else pth_rnd_gen_device

## Interprétabilité

##### Les jeux de données

In [ ]:
train_dataset, test_dataset, tokenizer, _ = get_datasets("./villes.txt")

##### Chargement du modèle de langue

In [ ]:
d_model = 32 # dimension du modèle
n_heads = 4 # nombre de têtes pour l'attention
n_layers = 1 # nombre de couches
dropout = 0.

batch_size = 64

In [ ]:
config = TransformerConfig(
    vocab_size=tokenizer.vocabulary_size(),
    d_model=d_model,
    n_heads=n_heads,
    n_layers=n_layers,
    dropout=dropout,
    max_len=max(train_dataset.max_len, test_dataset.max_len) - 1  # Because X and y : sequence[:-1] and sequence[1:] in dataset
)

filename = "./weights/model_32__4_heads__1_layers.pth" # A modifier selon le contexte
#filename = "`./weights/solutions/model_32__4_heads__1_layers.pth`" # A décommenter selon le contexte

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  target_dir_drive = '/content/drive/MyDrive'
  if Path(f"{target_dir_drive}/{repo}").exists() :
    filename_drive = filename.replace("/weights/", f"/{repo}/")
    !cp {target_dir_drive}/{filename_drive} {target_dir}/{filename}

In [ ]:
model = load_transformer_model(filename, class_model=LanguageModelForSAE, config=config, device=device)

##### Chargement du SAE

In [ ]:
act_size = config.d_model
num_features = 4 * config.d_model

In [ ]:
filename = "./weights/sae_model_32__4_heads__1_layers.pth" # A modifier selon le contexte
#filename = "./weights/solutions/sae_model_32__4_heads__1_layers.pt" # A décommenter selon le contexte

In [ ]:
if IN_COLAB:
  if Path(f"{target_dir_drive}/{repo}").exists() :
    filename_drive = filename.replace("/weights/", f"/{repo}/")
    !cp {target_dir_drive}/{filename_drive} {target_dir}/{filename}

In [ ]:
sae = load_sae(filename, act_size=act_size, num_features=num_features, device=device)

### Modifier le comportement en le dirigeant

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Récupération des valeurs maximum des activations de chaque caractéristique dans le SAE par rapport au jeu de données d'entrainement, afin de bénéficier d'un réfénrenciel pour y appliquer un facteur multiplicateur :

In [ ]:
model.eval();
sae.eval();

#min_values_sae = torch.full((sae.num_features, 1), +float('inf'))
max_values_sae = torch.full((sae.num_features, ), -float('inf'))

#Pour chaque élément du jeu de données d'entrainement
for X, _ in tqdm(train_dataloader, total=len(train_dataloader)):
    X = X.to(device)
    # récupération des activations cachées du Transformer
    hidden_acts_transfo = model(X, act=True) # (B, S, d_model)
    # récupération des activations de caractéristiques (features) correspondant
    _, _, features, _, _ = sae(hidden_acts_transfo) # (B, S, n_features)
    
    features = features.to("cpu")

    max_features, _ = features.max(dim=0)
    max_features, _ = max_features.max(dim=0)
    max_values_sae = torch.max(max_values_sae, max_features)

clean_memory(device)

Affichage de ces valeurs (valeur d'activation maximale par indice du neurone caché dans le SAE, sur le jeu de données):

In [ ]:
#print(max_values_sae)
for i, v in enumerate(max_values_sae):
    print(f"{i:3d}: {v:.2f}", end="   " if (i+1)%11 else "\n")

#### EXERCICES : génération contrôlée

Il s'agit d'appliquer la méthode de contrôle consistant à modifier les valeurs d'activations des neurones cachés dans le SAE, en connaissant les concepts qu'ils représentent, selon ce que l'on en a interprété.

![](https://drive.google.com/uc?id=1savRqhjCV4b36dWcQcEPUxouS1m_iftK)

![Contrôle avec SAE](./images/steering.png)

On remarque que l'erreur correspondant à la différence entre l'activation et sa reconstruction est exploitée dans le calcul du tenseur reconstruit en tenant compte des valeurs de contrôle.

Inspirez-vous largement du code de la fonction `sample()` de la partie 3 pour compléter cette fonction qui va générer des noms de commune en tenant compte des modifications qui sont founies via le tenseur `steering_vector`, contenant des lignes [`ìd neurone`, `modification`], selon la méthode illustrée dans le schéma ci-dessus.

Si le `max_values_sae` est fourni en paramètre, la valeur `modification` sera un facteur multiplicatif de la valeur maximale prise par le neurone d'indice `ìd neurone` présente dans `max_values_sae`.

> Conseil(s) : 
> - exploiter les méthodes d'indexation du type `steering_vector[:, 0]`, pour invoquer la première colonne d'un tenseur;
> - ne pas oublié de bien calculer l'erreur;

> NdA : vous constaterez après quelques tentatives d'utilisation de cette fonction que l'art de l'interprétabilité n'est pas une technique facilement maîtrisable.

In [ ]:
def steered_sample(
        model: LanguageModelForSAE,
        sae: AutoEncoder,
        tokenizer: CharTokenizer,
        steering_vector: torch.Tensor,
        prompt: str = "",
        max_values_sae: torch.Tensor = None,
        device="cpu",
        g = torch.Generator(),
        ) -> str:
    """
    Args:
        - model (LanguageModelForSAE) :
        - sae (AutoEncoder,) :
        - tokenizer (CharTokenizer) :
        - steering_vector (torch.Tensor) :
        - prompt (str = "") :
        - max_values_sae (torch.Tensor) :
        - device (str) :
        - g (torch.Generator) :
    """
    ### EXERCICE : compléter ce bloc avec les bonnes instructions 
    # DEBUT DE BLOC
    
    ### EXERCICE : à compléter

    return None ### EXERCICE : remplacer None par les bonnes instructions
    # DEBUT DE BLOC

Réalisons une génération dirigée de noms de commune, en imposant la valeur maximum du neurone ayant l'activation la plus forte :

In [ ]:
max_activation_index = max_values_sae.argmax().item()

steering_vector = torch.tensor([[max_activation_index, 1]], dtype=int, device="cpu")

for i in range(15):
    print(steered_sample(
        model,
        sae,
        tokenizer,
        prompt="la",
        steering_vector=steering_vector.to(device),
        max_values_sae=max_values_sae.to(device),
        device=device,
        g=pth_rnd_gen
        )
    )

A l'opposé, réalisons une génération dirigée de noms de commune, en neutralisant ce neurone :

In [ ]:
steering_vector = torch.tensor([[max_activation_index, 0]], dtype=int, device="cpu")

for i in range(15):
    print(steered_sample(
        model,
        sae,
        tokenizer,
        prompt="la",
        steering_vector=steering_vector.to(device),
        max_values_sae=max_values_sae.to(device),
        device=device,
        g=pth_rnd_gen
        )
    )

Comparons avec une génération non dirigée :

In [ ]:
for i in range(15):
    print(sample(
        model,
        tokenizer,
        prompt="la",
        device=device,
        g=pth_rnd_gen
        )
    )